In [1]:
#Handle Data
import pandas as pd
import numpy as np

#Handle Images
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#counter
from collections import Counter

#show plots and images
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score


In [2]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),          # Resize all images to 128x128
    transforms.ToTensor(),                  # Convert images to PyTorch tensors
    transforms.Normalize([0.5], [0.5])      # Normalize (adjust if 3-channel RGB)
])

In [3]:
dataset = datasets.ImageFolder('train', transform=transform)

# Split into DataLoader for batching
dataloader = DataLoader(dataset, batch_size=1000, shuffle=True)

In [4]:
# One batch of data
images, labels = next(iter(dataloader))

print(images.shape)  # (batch_size, channels, height, width)
print(labels)        # 0 for healthy, 1 for infected (automatically labeled based on folder names)


torch.Size([1000, 3, 128, 128])
tensor([ 0,  1,  3,  2,  6,  9,  7,  6,  8,  3,  9,  4,  5,  9,  0, 10,  9,  7,
         3,  9,  9,  7,  4,  3,  9, 10,  0,  1,  4,  8,  9,  7,  3,  4,  9,  8,
         5,  3,  4,  4,  9,  2,  9,  1,  4,  0,  7,  0,  9,  4,  6,  5,  4,  5,
         9,  5,  4,  3,  3,  5,  9,  9,  3,  0,  9,  3,  0,  8,  6,  1,  8,  7,
         6,  9,  6,  4,  3,  2, 10,  9,  9,  4,  1,  0,  9,  4,  3,  0,  3,  1,
         4,  7,  4,  6,  0,  0,  4,  1,  6,  0,  4,  4,  8, 10,  6,  7,  9,  9,
         6,  0,  4,  7,  0,  4,  9,  7,  8,  8,  3,  5,  3,  4, 10,  4,  4,  2,
         7,  9,  9,  8,  8,  4,  9,  7, 10,  1,  1,  9,  5,  0,  2,  4,  7,  3,
         0,  3, 10,  2,  4,  4,  7,  1,  0,  9,  4,  1,  8,  3,  3,  4,  7,  1,
         3,  9,  2,  0,  4,  3,  6,  0,  4,  7,  9,  4,  8,  8,  0,  0,  8,  6,
         2,  4,  9,  7,  4,  2,  4,  8,  4,  0,  2,  2,  4,  4,  8,  3,  0,  9,
         9,  2,  4,  0,  1,  1,  8,  4, 10,  6,  6,  0,  0,  2,  8,  5,  9,  4,
        

In [5]:
flattened = images.view(images.size(0), -1)
print(flattened.shape)  # (batch_size, 128*128*3) for RGB images
print(labels.shape)

torch.Size([1000, 49152])
torch.Size([1000])


In [18]:
from sklearn.decomposition import PCA

# Flatten images
X_flat = flattened

# Reduce dimensions (e.g., 50)
pca = PCA(n_components=17, whiten = True)
X_reduced = pca.fit_transform(X_flat)
X_reduced.shape

(1000, 17)

In [19]:
X = np.array(X_reduced)
y = np.array(labels)
X

array([[-0.46285692, -1.30733852, -0.74977217, ...,  0.15161866,
        -0.70770683, -1.69761843],
       [-0.41104739, -0.82628681, -0.35848729, ..., -0.13120145,
         0.76358009,  1.45156814],
       [ 0.00764804,  0.07636242, -1.28835126, ...,  0.5058502 ,
        -0.48842353,  0.90387943],
       ...,
       [-0.01377232, -0.99938258, -0.72667191, ...,  2.33967483,
         1.74080553, -0.03200791],
       [-0.20856355,  1.52324431,  1.70223906, ..., -1.13430001,
         0.85278774, -0.99773525],
       [-0.3020901 , -1.17453394,  0.62260826, ...,  0.27940719,
         0.41171059, -0.55462647]])

In [20]:
def standardization(X):
    X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

    return X

X = standardization(X)
X

array([[-0.46308852, -1.30799268, -0.75014734, ...,  0.15169452,
        -0.70806095, -1.69846788],
       [-0.41125306, -0.82670026, -0.35866667, ..., -0.1312671 ,
         0.76396217,  1.45229446],
       [ 0.00765186,  0.07640063, -1.28899592, ...,  0.50610331,
        -0.48866793,  0.90433171],
       ...,
       [-0.01377921, -0.99988264, -0.72703552, ...,  2.34084554,
         1.74167658, -0.03202393],
       [-0.20866791,  1.5240065 ,  1.70309081, ..., -1.13486759,
         0.85321446, -0.99823449],
       [-0.30224125, -1.17512165,  0.6229198 , ...,  0.279547  ,
         0.4119166 , -0.55490399]])

In [21]:
class_names = dataset.classes
class_names

['Bacterial_spot',
 'Early_blight',
 'Late_blight',
 'Leaf_Mold',
 'Septoria_leaf_spot',
 'Spider_mites Two-spotted_spider_mite',
 'Target_Spot',
 'Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato_mosaic_virus',
 'healthy',
 'powdery_mildew']

In [22]:
def standardization(X):
        X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    
        return X

class KNNClassifier:
    def __init__(self, k = 3):
        self.k = k

    def fit(self, train_x, train_y):
        self.train_x = standardization(np.array(train_x))
        self.train_y = np.array(train_y)

    def predict(self, test_x):
        self.test_x = standardization(np.array(test_x))

        predictions = []
        for x in self.test_x:
            dist = np.linalg.norm(self.train_x - np.array(x), axis = 1)
            min_indices = dist.argsort()[:self.k]
            min_ys = self.train_y[min_indices]
            most_common = Counter(min_ys).most_common()[0][0]
            predictions.append(most_common)
        return predictions
            

In [23]:
Counter([1,2,4,5,3,1,3,2,3]).most_common()[0][0]

3

In [24]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

In [25]:
model = KNNClassifier(k=6)
model.fit(X_train, y_train)
result = model.predict(X_test)

In [26]:
y_test

array([ 4,  3,  7,  6,  7,  8,  0,  9, 10,  1,  5,  3,  2, 10,  4,  5,  1,
        9,  0,  9,  8,  8,  2,  0,  0, 10,  4,  4,  4,  3,  4,  0,  8,  2,
        5,  9,  4,  7,  8,  5,  1,  4,  2,  8,  3,  0,  3,  1,  3,  1,  2,
        9,  4,  2,  3,  9,  0,  9,  9, 10,  6,  9,  2,  0,  9,  3,  2,  0,
        8,  0,  9,  2,  6,  2,  8,  2,  0,  4,  2,  0,  0,  0,  8,  8,  1,
        0,  2,  4,  4,  4,  2,  5,  8,  0,  0,  2,  8,  3,  0,  4,  9,  3,
        6,  3,  0,  0,  2,  0,  6,  4,  3,  1,  2,  3,  3,  3,  4,  8,  9,
        6,  3,  9,  5,  5,  1,  9,  3,  9,  4,  3,  0,  7,  9,  7,  9,  5,
        3,  4, 10,  6,  4,  5,  7,  0,  2,  3,  3,  4,  1,  0,  3,  3,  0,
        6,  5,  3,  3,  9,  9,  2,  4,  4,  0,  0,  9,  6,  0,  4,  0,  9,
        6,  0,  4,  4,  5,  8,  1,  3,  2,  9,  5,  0,  1,  5,  2,  9,  6,
        0,  9, 10,  2,  1,  9,  2,  5,  7,  1,  6,  2, 10], dtype=int64)

In [27]:
result

[6,
 7,
 7,
 6,
 7,
 7,
 0,
 8,
 1,
 0,
 1,
 4,
 2,
 9,
 2,
 0,
 2,
 8,
 2,
 9,
 8,
 8,
 2,
 4,
 0,
 4,
 0,
 9,
 4,
 6,
 4,
 0,
 8,
 2,
 0,
 4,
 0,
 6,
 5,
 1,
 0,
 4,
 2,
 5,
 2,
 0,
 4,
 0,
 3,
 1,
 9,
 6,
 8,
 6,
 1,
 6,
 0,
 4,
 9,
 2,
 0,
 9,
 2,
 2,
 1,
 9,
 4,
 0,
 5,
 6,
 4,
 2,
 4,
 3,
 9,
 1,
 7,
 7,
 2,
 2,
 0,
 0,
 8,
 9,
 0,
 2,
 4,
 1,
 2,
 10,
 8,
 5,
 2,
 9,
 4,
 0,
 8,
 3,
 7,
 8,
 9,
 4,
 4,
 2,
 0,
 0,
 2,
 2,
 5,
 4,
 3,
 0,
 9,
 4,
 1,
 3,
 10,
 8,
 5,
 0,
 3,
 2,
 5,
 7,
 8,
 9,
 9,
 9,
 3,
 3,
 0,
 0,
 9,
 7,
 6,
 8,
 3,
 0,
 6,
 0,
 4,
 5,
 7,
 0,
 9,
 4,
 3,
 4,
 1,
 7,
 4,
 1,
 0,
 2,
 8,
 3,
 8,
 9,
 10,
 9,
 2,
 2,
 2,
 9,
 5,
 0,
 7,
 4,
 9,
 8,
 8,
 0,
 5,
 9,
 5,
 1,
 7,
 3,
 9,
 0,
 1,
 2,
 7,
 8,
 9,
 9,
 6,
 10,
 4,
 9,
 5,
 4,
 2,
 1,
 5,
 7,
 8,
 8,
 2,
 2]

In [28]:
p = 0
for i in range(len(y_test)):
    if y_test[i] == result[i]:
        p = p + 1

p/len(y_test)

0.345

In [29]:
for k in range(1, 30):
    model = KNNClassifier(k=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"k = {k}, accuracy = {acc:.4f}")

k = 1, accuracy = 0.3400
k = 2, accuracy = 0.3400
k = 3, accuracy = 0.3400
k = 4, accuracy = 0.3750
k = 5, accuracy = 0.3550
k = 6, accuracy = 0.3450
k = 7, accuracy = 0.3550
k = 8, accuracy = 0.3500
k = 9, accuracy = 0.3400
k = 10, accuracy = 0.3450
k = 11, accuracy = 0.3500
k = 12, accuracy = 0.3600
k = 13, accuracy = 0.3400
k = 14, accuracy = 0.3350
k = 15, accuracy = 0.3450
k = 16, accuracy = 0.3450
k = 17, accuracy = 0.3350
k = 18, accuracy = 0.3600
k = 19, accuracy = 0.3450
k = 20, accuracy = 0.3450
k = 21, accuracy = 0.3600
k = 22, accuracy = 0.3650
k = 23, accuracy = 0.3400
k = 24, accuracy = 0.3400
k = 25, accuracy = 0.3300
k = 26, accuracy = 0.3200
k = 27, accuracy = 0.3200
k = 28, accuracy = 0.3200
k = 29, accuracy = 0.3200
